In [ ]:
import time
import gazepoint
from reachy import Leachy
from pypot.robot.remote import from_remote
from pypot.creatures import ik
from ikpy.chain import Chain
import numpy

In [ ]:
leachy = from_remote('leachy.local', 4242)

In [ ]:
leachy.motors

In [ ]:
import os
urdf = os.path.join(os.getcwd(), 'reachy', 'software', 'reachy', 'leachy.urdf')

In [ ]:
leachy_kinematic = Chain.from_urdf_file(urdf, base_elements=['base'], last_link_vector=[0, 0, 0])

In [ ]:
x_amp = 0.1 #m
y_amp = 0.1 #m
x_center = 0 #m
y_center = -0.3 #m 

def gaze2target():
    [x, y] = gazetracker.get_gaze_position()
    
    # gaze to motion
    x = - (1 - max(min(x, 1), 0)) * x_amp
    y = (1 - max(min(y, 1), 0)) * y_amp
    
    # IK target
    goal = [0.3, x_center - x_amp/2.0 + x, y_center - y_amp/2.0 + y]
    target = numpy.eye(4)
    target[:3, :3] = numpy.eye(3)
    target[:3, 3] = goal
    
    return target

rest_pos = [0] + [numpy.deg2rad(m.present_position) for m in leachy.motors] + [0]

def follow_gaze():
    target = gaze2target()
    # start from current position:
    start_pos = [0] + [numpy.deg2rad(m.present_position) for m in leachy.motors] + [0]
    # start from defined position:
    #start_pos = rest_pos
    
    joint_position = leachy_kinematic.inverse_kinematics(target, start_pos, max_iter=3)
    
    for m, p in zip(leachy.motors, joint_position[1: -1]):
        m.goal_position = numpy.rad2deg(p)

In [ ]:
gazetracker = gazepoint.GazePoint()

In [ ]:
for m in leachy.motors:
    m.compliant = True
    m.moving_speed = 50

In [ ]:
while True:
    follow_gaze()

In [ ]:
leachy.compliant = True